Infiltrationsbestimmung nach Green und Ampt

Mit dem Green-Ampt Modell lässt sich die Infiltrationsrate in eine Bodensäule mit einem Querschnitt von 1 m² als exponentielle Funktion der Zeit darstellen.

$ Formel §

In [53]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
from ipywidgets import interactive_output
from IPython.display import clear_output
from ipywidgets import GridspecLayout


Messdaten für die Infiltration [l/s] in einen 1 m² großen Bodenquerschnitt können als csv-Datei durch Eingabe des vollständigen Dateipfades in das nachstehende Eingabefenster eingelesen werden.

Im zweiten Eingabefenster ist der Spaltenname der Zeitdatenreihe einzufügen.

In [54]:


class GreenAmptModel:                                                         # Klasse mit Methoden zur Berechnung und Visualisierung der Infiltration nach Green-Ampt (1911)

    def __init__(self):             
        self.reference_lines = []                                                               # Initialisierung von Parametern bzw. Listen, in denen später berechnete
        self.ref_line_params = []                                                               # Werte gespeichert werden können.
        self.short_time_ref = []                                                                # Durch die Initialisierung sind die mit "self" markierten Variablen für alle
        self.short_time_params = []                                                             # Methoden in der Klasse: GreenAmptModel sichtbar.
        self.infiltration_rate_ref = []
        self.infiltration_ref = []


    def t_for_It(self, I_t, d_theta, d_h, Ks=0.3):                                              # Methode zur Berechnung der Zeitschritte zwischen einzelnen Infiltrationsschrittenb(t)
            return (I_t - d_theta * d_h * np.log(1+ (I_t/(d_theta*d_h))))/Ks                            # Ausgabe: np.array

    def long_time_approximation(self, K, x):                                                    # Methode zur Berechnung der Langzeitannäherung (i_t_long) an die Infiltrationsrate (i_t)
        it_long = np.array([K]*len(x))                                                                  # Ausgabe: np.array
        return it_long

    def d_h(self, h0=0, hf=-60):                                                                # Methode zur Berechnung einer Differenz zwischen zwei Werten
        return h0-hf                                                                                    # Ausgabe: Float

    def d_theta(self, theta_0=0.44, theta_i=0.1):
        return theta_0 - theta_i
    
    def infiltration_rate(self, new_t, t, I_t):                                                 # Methode zur Berechnung der Infiltationsate 
        y2 = np.interp(new_t, t, I_t)                                                                   # Ausgabe: np.array
        i_t = np.diff(y2)
        #self.i_t = i_t
        return i_t
    
    def read_datastring(self, y_data_string='', x_data_string=''):                              # Methode zum Einlesen von Messdaten
        if y_data_string or x_data_string != '':                                                        # Die Messdaten werden als String eingelesen und in Listen umgewandelt
            y_data = y_data_string.split(',')                                                           # Ausgabe: 2 Listen mit Funktionswerten
            x_data = y_data_string.split(',')
            return y_data, x_data


    def plot_scenario(                                                                          # Methode zum Plotten der Daten
            self, hf_h0=(-60,0), thetai_theta0=(0.2,0.44), Ks=4, i_data_string='',
            ti_data_string='', I_data_string='', tI_data_string='' 
            ):


        h0 = hf_h0[1]                                                                                   # Vom Benutzer eingelesene Werte werden in Variablen gespeichert
        hf = hf_h0[0]
        theta_i = thetai_theta0[0]
        theta_0 = thetai_theta0[1]
        delta_theta = self.d_theta(theta_0, theta_i)
        delta_h = self.d_h(h0, hf)
        
                                                                                                        # Aus den definierten Methoden werden Daten generiert
        I_t = np.arange(0.1, 260, 0.1)                                                                      # Infiltrationsdaten (type: np.array)
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks)                                                    # Zeitschritte für die Infiltration (type: np.array)
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)                                       # Zeitschritte für die Infiltration auf 1. Dezimalstelle gerundet (type: np.array)
        self.i_t = self.infiltration_rate(self.new_t, t, I_t)                                               # Infiltrationsrate (type: np.array)
        i_t_long = self.long_time_approximation(Ks, self.new_t)                                             # Langzeitannäherung der Infiltrationsrate ~ Ks (type: np.array)


        fig, (ax1, ax2) = plt.subplots(2, figsize=(12,12))                                              # Eine Abbildung für die Plots wird erstellt

        # Für den Button: Plotten der Referenzlinien, der gespeicherten Graphen
        for ref, params in zip(self.infiltration_ref, self.ref_line_params):                            # Unten definierte Referenzlinien für I_t werden geplottet
            ref_x, ref_y = ref
            ax1.plot(ref_x, ref_y, linestyle='-.', alpha=0.6, 
                     label='hf:{} h0:{} theta_i:{} theta_0:{} Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)
        
        for ref, params in zip(self.infiltration_rate_ref, self.ref_line_params):                       # Unten definierte Referenzlinien für i_t werden geplottet
            ref_x, ref_y = ref
            ax2.plot(ref_x, ref_y, linestyle='-.', alpha=0.6,
                      label='hf:{} h0:{} theta_i:{} theta_0:{} Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)
        
        # Plotten der Daten
        ax1.plot(t, I_t, color='black', label='I(t)', linewidth=1)
        
        ax2.plot(self.new_t[1:-5], self.i_t[0:-5]*100, color='black', label='i(t)', linewidth=1)
        ax2.plot(self.new_t, i_t_long, 'red', linestyle='--', label='Long time approximation ~ Ks [mm/min]')

        # Formatierung
        ax1.set_title(label='Infiltration', fontsize=20, loc= 'center')
        ax1.set_ylim(0.1, max(I_t))
        ax1.set_xlim(0, 185)
        ax1.set_xlabel('Zeit [min]', fontsize=10)
        ax1.set_ylabel('Infiltration [mm/min]', fontsize=10)

        ax1.set_title(label='Infiltrationsrate', fontsize=20, loc= 'center')
        ax2.set_ylim(0.1, 150)
        ax2.set_xlim(0, 185)
        ax2.set_yscale('log')
        ax2.set_xlabel('Zeit [min]', fontsize=10)
        ax2.set_ylabel('Infiltrationsrate [mm/min]', fontsize=10)


        # Platzieren der Legende
        ax1.legend(loc='best')
        ax2.legend(loc='best')

        # Plotten von zusätzlichen, vom Benutzer eingegebenen Messdaten
        ## für die Infiltrationsrate
        if i_data_string or ti_data_string != '':                                                 # Wurden Messdaten übergeben? Wenn, ja:
            i_data, ti_data = self.read_datastring(i_data_string, ti_data_string)                 # Umwandeln von str -> list
            ax2.plot(ti_data, i_data, color='blue', linestyle='', marker='+')                     # Plotten

        ## für die Infiltration
        if I_data_string or tI_data_string != '':                                                 # Wurden Messdaten übergeben? Wenn, ja:
            I_data, tI_data = self.read_datastring(I_data_string, tI_data_string)                 # Umwandeln von str -> list
            ax1.plot(tI_data, I_data, color='blue', linestyle='', marker='+')                     # Plotten

        plt.show()

    def on_button_click(self, button):
        # Speichern der über "Widgets" eingelesenen Werte in Variablen
        h0_val = self.hf_h0.value[1]
        hf_val = self.hf_h0.value[0]
        theta_i_val = self.thetai_theta0.value[0]
        theta_0_val = self.thetai_theta0.value[1]
        Ks_val = self.Ks_slider.value

        delta_theta = (self.d_theta(theta_0_val, theta_i_val))
        delta_h = self.d_h(h0_val, hf_val)

        I_t = np.arange(0, 260, 0.1)
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks_val)
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)
        #i_t_short = self.short_time_approximation(delta_theta, delta_h, Ks_val, self.new_t)
        i_t = self.infiltration_rate(self.new_t, t, I_t)
        print('t', len(t), 'i_t', len(i_t))

        # Store the current short time approximation data as a reference line
        self.reference_lines.append((t, I_t))
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))
        self.reference_lines.append((self.new_t, i_t))
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))
        #self.short_time_ref.append((self.new_t, i_t))
        #self.short_time_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))

        #it = self.i_t(self.new_t, t, I_t)
        self.infiltration_ref.append((t, I_t))
        self.infiltration_rate_ref.append((self.new_t[1:-5], self.i_t[:-5]*100))

        clear_output(wait=True)

        #display(self.grid)
        #self.plot_scenario()
        display(self.interactive_plot)
        display(button)

        #display(widgets.VBox(children=[tab, button]))


    def create_interactive_plot(self):
        # Create the button
        button = Button(description="Fixate Short time approximation")
        button.on_click(self.on_button_click)

        # Create sliders for the parameters
        self.hf_h0 = widgets.FloatRangeSlider(
            value=[-600, 0], min=-1000, max=100, step=0.1, description=('hf - h0 [mm]')
            )
        self.thetai_theta0 = widgets.FloatRangeSlider(
            value=[0.1, 0.4], min=0, max=0.5, step=0.01, description=('theta i - theta 0 []')
            )
        self.Ks_slider = widgets.FloatSlider(
            value=0.5, min=0.01, max=5, step=0.01, description='Ks [mm/min]' 
            )

        # Sliders for reading in data 
        self.I_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
            description='Infiltrationsmessdaten'
            )
        
        self.tI_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Zeitdaten für I(t)'
              )

        self.i_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Infiltrationsrate'
              )
        
        self.ti_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Zeitdaten für i(t)'
              )


        '''
        tab1 = VBox(children=[self.hf_h0, self.thetai_theta0, self.Ks_slider])
        tab2 = VBox(children=[self.I_data_string, self.tI_data_string])
        tab3 = VBox(children=[self.i_data_string, self.ti_data_string])

        tab = widgets.Tab(children=[tab1, tab2, tab3])
        tab.set_title(0, 'Variablen')
        tab.set_title(1, 'Messdaten I(t)')
        tab.set_title(2, 'Messdaten i(t)')

        display(widgets.VBox(children=[tab, button]))
        '''

        # Forat Widgets
        self.grid = GridspecLayout(6,3, hight='300px')
        self.grid[:2, :1] = self.hf_h0
        self.grid[2:4, :1] = self.thetai_theta0
        self.grid[4:6, :1] = self.Ks_slider
        self.grid[:3, 1:2] = self.i_data_string
        self.grid[3:6, 1:2] = self.ti_data_string
        self.grid[:3, 2:3] = self.I_data_string
        self.grid[3:6, 2:3] = self.tI_data_string
        #display(self.grid)


        # Display the interactive plot with the sliders and button
        self.interactive_plot = interactive(self.plot_scenario, hf_h0 = self.hf_h0, thetai_theta0 = self.thetai_theta0, Ks=self.Ks_slider, i_data_string=self.i_data_string, ti_data_string=self.ti_data_string, I_data_string=self.I_data_string, tI_data_string=self.tI_data_string)
        #self.interactive_plot
        #self.plot_scenario()
        #display(self.grid)


        display(self.interactive_plot)        
        display(button)



In [55]:
GreenAmptModel().create_interactive_plot()


interactive(children=(FloatRangeSlider(value=(-600.0, 0.0), description='hf - h0 [mm]', layout=Layout(grid_are…

Button(description='Fixate Short time approximation', style=ButtonStyle())